In [35]:
import pandas as pd
import seaborn as sns
data = pd.read_csv("df.csv")
data.head()

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,Sector
0,2Q3DU,1.19,0.20,2.21,0.0770,6.66,0.40,0.43,1.95,5.93,Industrials
1,46JNq,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,Communication Services
2,4ISOt,2.93,0.04,1.43,0.3055,6.55,0.72,0.06,7.78,18.38,Communication Services
3,5AWwu,1.03,0.25,3.55,0.0708,146.43,1.80,0.88,6.17,5.15,Consumer Defensive
4,5hbZR,1.20,0.47,0.00,0.1677,30.92,0.93,0.00,0.00,17.06,Consumer Cyclical


data o hodnotách finančních indikátorů 100 největších společností obchodovaných na americké burze. Naším cílem je zjistit, které indikátory nejvíce ovlivňují cenu, a vytvořit model, který odhadne cenu akcie na základě hodnot finančních indikátorů.

Jeden z indikátorů je označený jako Y a je poměrem ceny a účetní hodnoty akcie. Hodnota indikátoru je odrazem ceny akcie. Ostatní indikátory jso následující:

běžná likvidita (Current Ratio, CR),
zadluženost (Debt to Assets, DA),
finanční páka (Financial Leverage, FL)
provozní zisková marže (Operating Profit Margin, OPM),
obrat pohledávek (Receivables Turnover, RT),
obrat celkových aktiv (Total Assets Turnover, TAT).

Vytvoř korelační matici a podívej se, který ukazatel má největší vliv na indikátor Y akcie.

In [36]:
data_corr = data.drop(columns=["Company", "Sector"])

In [37]:
data_corr.corr()

,CR,DA,FL,OPM,RT,TAT,DE,Y,PT
CR,1.000000,-0.225917,-0.179232,0.284529,-0.151203,-0.123665,-0.141136,-0.018690,-0.040483
DA,-0.225917,1.000000,0.340320,0.015032,0.206620,-0.083691,0.464786,0.164599,-0.120993
FL,-0.179232,0.340320,1.000000,-0.112430,-0.076335,0.156169,0.968612,0.759323,0.032588
OPM,0.284529,0.015032,-0.112430,1.000000,0.051672,-0.298701,-0.050014,0.051308,0.021409
RT,-0.151203,0.206620,-0.076335,0.051672,1.000000,0.206580,-0.065267,-0.046290,0.097708
TAT,-0.123665,-0.083691,0.156169,-0.298701,0.206580,1.000000,0.094869,0.301222,0.017872
DE,-0.141136,0.464786,0.968612,-0.050014,-0.065267,0.094869,1.000000,0.724604,-0.025348
Y,-0.018690,0.164599,0.759323,0.051308,-0.046290,0.301222,0.724604,1.000000,-0.001363
PT,-0.040483,-0.120993,0.032588,0.021409,0.097708,0.017872,-0.025348,-0.001363,1.000000


odpověď: Největší "vliv"má akcie FL. 

Vytvoř regresní model, který bude mít koeficient Y jako vysvětlovanou proměnnou. Do modelu vlož hodnoty ostatních indikátorů jako vysvětlující proměnné.

In [38]:
import statsmodels.formula.api as smf
formula = "Y ~ CR + DA + FL + OPM + RT + TAT + DE + PT"
mod = smf.ols(formula=formula, data=data_corr)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.635
Method:                 Least Squares   F-statistic:                     22.52
Date:                Thu, 16 Nov 2023   Prob (F-statistic):           1.45e-18
Time:                        18:41:14   Log-Likelihood:                -375.94
No. Observations:                 100   AIC:                             769.9
Df Residuals:                      91   BIC:                             793.3
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.0176      5.347     -2.061      0.042     -21.638      -0.397
CR             1.5014      1.240      1.211      0.229      -0.961       3.964
DA            -5.2206      9.370     -0.557      0.579     -23.834      13.392
FL             3.0104      1.060      2.840      0.006       0.905       5.116
OPM           23.4665      8.076      2.906      0.005       7.425      39.508
RT            -0.0063      0.021     -0.300      0.765      -0.048       0.035
TAT            8.4254      2.351      3.584      0.001       3.756      13.095
DE            -0.7283      2.066     -0.352      0.725      -4.833       3.376
PT            -0.0644      0.096     -0.669      0.505      -0.256       0.127
==============================================================================
Omnibus:                       29.136   Durbin-Watson:                   1.900
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              101.551
Skew:                           0.881   Prob(JB):                     8.88e-23
Kurtosis:                       7.611   Cond. No.                         592.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

S využitím modulu statsmodels vytvoř regresní model a zobraz si tabulku se souhrnem významů. Podívej se na hodnoty koeficientů a na výsledky testu statistické významnosti koeficientů. Pokud je některý koeficient (nebo více koeficientů) nevýznamný, sestav nový model bez tohoto koeficientů (případně beze všech nevýznamných koeficientů).

In [39]:
formula = "Y ~ FL + OPM + TAT"
mod = smf.ols(formula=formula, data=data_corr)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.650
Model:                            OLS   Adj. R-squared:                  0.639
Method:                 Least Squares   F-statistic:                     59.51
Date:                Thu, 16 Nov 2023   Prob (F-statistic):           7.99e-22
Time:                        18:41:19   Log-Likelihood:                -377.99
No. Observations:                 100   AIC:                             764.0
Df Residuals:                      96   BIC:                             774.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.4188      2.865     -3.637      0.000     -16.106      -4.732
FL             2.5281      0.208     12.149      0.000       2.115       2.941
OPM           24.8478      7.542      3.295      0.001       9.878      39.818
TAT            8.5518      2.205      3.879      0.000       4.176      12.928
==============================================================================
Omnibus:                       26.748   Durbin-Watson:                   1.921
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               76.291
Skew:                           0.876   Prob(JB):                     2.71e-17
Kurtosis:                       6.904   Cond. No.                         50.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Pro všechna data odhadni ukazatel Y s využitím tvého modelu a odhadnuté ceny vlož do původní tabulky s daty. Dále vypočítej rozdíl mezi odhadem koeficientu a jeho skutečnou hodnotou. Najdi akcii, kde je tento rozdíl největší (tj. hledáme akcii, které náš model predikuje výrazně vyšší cenu než jaká je ve skutečnosti, tato akcie je potenciálně na trhu podhodnocená).

In [40]:
data_to_predict = pd.read_csv("df.csv")
data_to_predict["Y_predicted"] = res.predict(data_to_predict)
data_to_predict["real_predicted_diff"] = data_to_predict["Y_predicted"] - data_to_predict["Y"]
data_to_predict.sort_values(by=["real_predicted_diff"],ascending=False)

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,Sector,Y_predicted,real_predicted_diff
79,SYXLy,0.76,0.37,18.64,0.0631,12.44,0.41,6.93,6.97,3.40,Industrials,41.779545,34.809545
98,YX515,2.30,0.64,25.03,0.3758,7.48,0.31,16.08,41.57,10.54,Technology,64.848968,23.278968
6,6qLRA,1.19,0.49,13.56,-0.0415,14.81,0.36,6.69,3.06,4.63,Industrials,25.910015,22.850015
1,46JNq,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,Communication Services,23.356512,16.096512
56,n2wGj,1.00,0.14,3.37,0.0349,106.52,3.24,0.49,11.30,10.49,Consumer Defensive,26.676012,15.376012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,mS0GZ,1.38,0.11,2.06,0.1212,28.14,0.87,0.23,31.06,4.01,Consumer Cyclical,5.240760,-25.819240
81,tX5FA,1.05,0.15,2.92,0.0436,4.24,0.55,0.43,30.36,15.20,Technology,2.750181,-27.609819
39,hqTVU,1.29,0.37,5.15,0.5339,14.32,0.50,1.90,49.43,20.68,Financial Services,20.143170,-29.286830
75,QiwB1,1.23,0.53,21.39,0.1384,44.15,1.87,11.29,95.58,7.52,Consumer Cyclical,63.088553,-32.491447


odpověď: Akcie 79

Bonus 1: Sestav model s využitím robustní regrese. Opět proveď vyřazení koeficinetů, které nejsou statisticky významné, a sestav model pouze s významnými koeficienty. Vlož odhady cen do původních dat a opět najdi potenciálně nejvíce podhodnocenou akcii. Jde v případě robustní regese o stejnou akcii, nebo se akcie liší?

In [10]:
formula = "Y ~ CR + DA + FL + OPM + RT + TAT + DE + PT"
mod = smf.rlm(formula=formula, data=data_corr)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Robust linear Model Regression Results                    
==============================================================================
Dep. Variable:                      Y   No. Observations:                  100
Model:                            RLM   Df Residuals:                       91
Method:                          IRLS   Df Model:                            8
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Wed, 15 Nov 2023                                         
Time:                        23:21:09                                         
No. Iterations:                    37                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.4961      3.453     -3.039      0.002     -17.265      -3.728
CR             2.0316      0.801      2.537      0.011       0.462       3.601
DA            -0.4762      6.052     -0.079      0.937     -12.338      11.386
FL             3.6694      0.685      5.360      0.000       2.328       5.011
OPM           16.3810      5.216      3.140      0.002       6.158      26.604
RT            -0.0029      0.014     -0.216      0.829      -0.029       0.024
TAT            5.2670      1.518      3.469      0.001       2.291       8.243
DE            -2.5507      1.335     -1.911      0.056      -5.166       0.065
PT            -0.0878      0.062     -1.412      0.158      -0.210       0.034
==============================================================================

If the model instance has been used for another fit with different fit parameters, then the fit options might not be the correct ones anymore .
"""

In [11]:
formula = "Y ~ CR + FL + OPM + TAT"
mod = smf.rlm(formula=formula, data=data_corr)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Robust linear Model Regression Results                    
==============================================================================
Dep. Variable:                      Y   No. Observations:                  100
Model:                            RLM   Df Residuals:                       95
Method:                          IRLS   Df Model:                            4
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Wed, 15 Nov 2023                                         
Time:                        23:22:47                                         
No. Iterations:                    23                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -9.9579      1.924     -5.176      0.000     -13.729      -6.187
CR             2.0977      0.681      3.081      0.002       0.763       3.432
FL             2.4422      0.123     19.840      0.000       2.201       2.683
OPM           13.9913      4.560      3.068      0.002       5.054      22.929
TAT            5.3858      1.290      4.176      0.000       2.858       7.913
==============================================================================

If the model instance has been used for another fit with different fit parameters, then the fit options might not be the correct ones anymore .
"""

In [12]:
data_to_predict = pd.read_csv("df.csv")
data_to_predict["Y_predicted"] = res.predict(data_to_predict)
data_to_predict["real_predicted_diff"] = data_to_predict["Y_predicted"] - data_to_predict["Y"]
data_to_predict.sort_values(by=["real_predicted_diff"],ascending=False)

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,Sector,Y_predicted,real_predicted_diff
79,SYXLy,0.76,0.37,18.64,0.0631,12.44,0.41,6.93,6.97,3.40,Industrials,40.250244,33.280244
6,6qLRA,1.19,0.49,13.56,-0.0415,14.81,0.36,6.69,3.06,4.63,Industrials,27.013046,23.953046
98,YX515,2.30,0.64,25.03,0.3758,7.48,0.31,16.08,41.57,10.54,Technology,62.922966,21.352966
1,46JNq,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,Communication Services,20.203407,12.943407
24,Cdr0m,1.26,0.24,3.14,0.0187,11.43,1.97,0.75,1.91,8.87,Energy,11.225472,9.315472
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,mS0GZ,1.38,0.11,2.06,0.1212,28.14,0.87,0.23,31.06,4.01,Consumer Cyclical,4.349329,-26.710671
81,tX5FA,1.05,0.15,2.92,0.0436,4.24,0.55,0.43,30.36,15.20,Technology,2.948199,-27.411801
39,hqTVU,1.29,0.37,5.15,0.5339,14.32,0.50,1.90,49.43,20.68,Financial Services,15.488436,-33.941564
75,QiwB1,1.23,0.53,21.39,0.1384,44.15,1.87,11.29,95.58,7.52,Consumer Cyclical,56.869082,-38.710918


odpověď: Opět je to akcie 79.

Bonus 2: Použij původní (tedy "nerobustní") model a vyčísli Cookovu vzdálenost pro všechny hodnoty. Vyřaď všechny akcie s Cookovou vzdáleností vyšší než 1 a přepočítej regresní model. Nakonec opět najdi potenciálně nejvíce podhodnocenou akcii a podívej se, jestli jde o stejnou akcii jako u předchozích modelů.

In [16]:
from statsmodels.stats.outliers_influence import OLSInfluence
formula = "Y ~ FL + OPM + TAT"
mod = smf.ols(formula=formula, data=data_corr)
results = mod.fit()
influence = OLSInfluence(results)
data_corr['Cook Distance'] = influence.cooks_distance[0]
data_corr = data_corr[data_corr['Cook Distance'] < 1]
mod = smf.ols(formula=formula, data=data_corr)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.650
Model:                            OLS   Adj. R-squared:                  0.639
Method:                 Least Squares   F-statistic:                     59.51
Date:                Wed, 15 Nov 2023   Prob (F-statistic):           7.99e-22
Time:                        23:32:05   Log-Likelihood:                -377.99
No. Observations:                 100   AIC:                             764.0
Df Residuals:                      96   BIC:                             774.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.4188      2.865     -3.637      0.000     -16.106      -4.732
FL             2.5281      0.208     12.149      0.000       2.115       2.941
OPM           24.8478      7.542      3.295      0.001       9.878      39.818
TAT            8.5518      2.205      3.879      0.000       4.176      12.928
==============================================================================
Omnibus:                       26.748   Durbin-Watson:                   1.921
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               76.291
Skew:                           0.876   Prob(JB):                     2.71e-17
Kurtosis:                       6.904   Cond. No.                         50.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
data_to_predict = pd.read_csv("df.csv")
data_to_predict["Y_predicted"] = res.predict(data_to_predict)
data_to_predict["real_predicted_diff"] = data_to_predict["Y_predicted"] - data_to_predict["Y"]
data_to_predict.sort_values(by=["real_predicted_diff"],ascending=False)

,Company,CR,DA,FL,OPM,RT,TAT,DE,Y,PT,Sector,Y_predicted,real_predicted_diff
79,SYXLy,0.76,0.37,18.64,0.0631,12.44,0.41,6.93,6.97,3.40,Industrials,41.779545,34.809545
98,YX515,2.30,0.64,25.03,0.3758,7.48,0.31,16.08,41.57,10.54,Technology,64.848968,23.278968
6,6qLRA,1.19,0.49,13.56,-0.0415,14.81,0.36,6.69,3.06,4.63,Industrials,25.910015,22.850015
1,46JNq,0.29,0.64,10.14,0.2037,20.04,0.36,6.52,7.26,43.48,Communication Services,23.356512,16.096512
56,n2wGj,1.00,0.14,3.37,0.0349,106.52,3.24,0.49,11.30,10.49,Consumer Defensive,26.676012,15.376012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,mS0GZ,1.38,0.11,2.06,0.1212,28.14,0.87,0.23,31.06,4.01,Consumer Cyclical,5.240760,-25.819240
81,tX5FA,1.05,0.15,2.92,0.0436,4.24,0.55,0.43,30.36,15.20,Technology,2.750181,-27.609819
39,hqTVU,1.29,0.37,5.15,0.5339,14.32,0.50,1.90,49.43,20.68,Financial Services,20.143170,-29.286830
75,QiwB1,1.23,0.53,21.39,0.1384,44.15,1.87,11.29,95.58,7.52,Consumer Cyclical,63.088553,-32.491447


odpověď: Opět je to stejná akcie 79.

Bonus 3: Přidej Sektor (poslední sloupec) jako vysvětlující proměnnou s využitím One Hot Encoding. Podívej se, které sektory mají kladný koeficient a které sektory mají záporný koeficient.

In [41]:
data = pd.read_csv("df.csv")
sector = pd.get_dummies(data["Sector"])
data = pd.merge(data, sector, left_index=True, right_index=True)


In [30]:
data.columns #zde zjišťuji, že mi regresní rovnice nevezme názvy proměnných s mezerou, takže je upravuji, aby místo mezery bylo podtržítko

Index(['Company', 'CR', 'DA', 'FL', 'OPM', 'RT', 'TAT', 'DE', 'Y', 'PT',
       'Sector', 'Basic Materials', 'Communication Services', 'Conglomerates',
       'Consumer Cyclical', 'Consumer Defensive', 'Energy',
       'Financial Services', 'Healthcare', 'Industrials', 'Technology'],
      dtype='object')

In [31]:
data.columns = [c.replace(' ', '_') for c in data.columns]
data.columns

Index(['Company', 'CR', 'DA', 'FL', 'OPM', 'RT', 'TAT', 'DE', 'Y', 'PT',
       'Sector', 'Basic_Materials', 'Communication_Services', 'Conglomerates',
       'Consumer_Cyclical', 'Consumer_Defensive', 'Energy',
       'Financial_Services', 'Healthcare', 'Industrials', 'Technology'],
      dtype='object')

In [32]:
formula = "Y ~ FL + OPM + TAT + Basic_Materials + Communication_Services + Conglomerates + Consumer_Cyclical + Consumer_Defensive + Energy + Financial_Services + Healthcare + Industrials + Technology"
mod = smf.ols(formula=formula, data=data)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.702
Model:                            OLS   Adj. R-squared:                  0.661
Method:                 Least Squares   F-statistic:                     17.08
Date:                Thu, 16 Nov 2023   Prob (F-statistic):           4.32e-18
Time:                        18:30:42   Log-Likelihood:                -369.98
No. Observations:                 100   AIC:                             766.0
Df Residuals:                      87   BIC:                             799.8
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -6.9599      2.967     -2.346      0.021     -12.857      -1.063
FL                         2.4726      0.208     11.873      0.000       2.059       2.887
OPM                       17.6464      8.265      2.135      0.036       1.219      34.073
TAT                        7.4755      2.424      3.084      0.003       2.658      12.293
Basic_Materials            0.2567      5.671      0.045      0.964     -11.015      11.528
Communication_Services    -3.1696      3.372     -0.940      0.350      -9.873       3.533
Conglomerates             -3.8534      9.604     -0.401      0.689     -22.942      15.236
Consumer_Cyclical          3.9228      3.208      1.223      0.225      -2.454      10.299
Consumer_Defensive        -1.5535      3.698     -0.420      0.675      -8.904       5.797
Energy                    -5.4862      4.169     -1.316      0.192     -13.772       2.799
Financial_Services         9.7416      6.040      1.613      0.110      -2.263      21.747
Healthcare                -2.7452      2.642     -1.039      0.302      -7.997       2.506
Industrials               -7.1937      2.991     -2.405      0.018     -13.139      -1.249
Technology                 3.1205      2.626      1.188      0.238      -2.099       8.340
==============================================================================
Omnibus:                       30.481   Durbin-Watson:                   1.833
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              103.310
Skew:                           0.945   Prob(JB):                     3.69e-23
Kurtosis:                       7.607   Cond. No.                     1.13e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.7e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

odpověď: Záporné koeficienty mají sektory Communication Services, Conglomerates, Consumer Defensive, Energy, Healthcare a Industrials.  